In [ ]:
from pyspark.sql.functions import col, expr

# Read from Delta table
weather_stats = spark.read.table("weather_stats")

# Create temporary view for SQL queries
weather_stats.createOrReplaceTempView("weather_stats")

# Example analysis queries
# 1. Find temperature anomalies
anomalies = spark.sql("""
    SELECT 
        sensor_id,
        window,
        temperature_avg,
        humidity_avg,
        wind_speed_avg
    FROM weather_stats
    WHERE temperature_avg > (
        SELECT AVG(temperature_avg) + 2 * STDDEV(temperature_avg)
        FROM weather_stats
    )
""")

# 2. Calculate sensor reliability
sensor_reliability = spark.sql("""
    SELECT 
        sensor_id,
        COUNT(*) as reading_count,
        AVG(temperature_avg) as avg_temperature,
        STDDEV(temperature_avg) as temp_stability
    FROM weather_stats
    GROUP BY sensor_id
    ORDER BY reading_count DESC
""")

# Display results using Databricks display() function
display(anomalies)
display(sensor_reliability)